# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,juan griego,11.0817,-63.9655,31.07,74,52,6.69,VE,1720303731
1,1,namie,37.4833,141.0000,29.29,76,92,1.35,JP,1720303732
2,2,kourou,5.1552,-52.6478,28.03,84,99,3.51,GF,1720303734
3,3,dudinka,69.4058,86.1778,8.71,93,81,4.70,RU,1720303735
4,4,lompoc,34.6391,-120.4579,33.08,53,9,2.40,US,1720303569


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    frame_width = 700,
    frame_height = 500,
    scale = 0.75,
    color = 'City',
    alpha = 0.5
)
city_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# max temp lower than 27 degrees and above 21 degrees C
# wind speed less than 4.5 m/s
# cloudiness = 0

city_data_df = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27)]
city_data_df = city_data_df[(city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,kandahar,31.0000,65.7500,25.71,14,0,1.99,AF,1720303745
116,116,prince george,53.9166,-122.7530,25.72,41,0,0.89,CA,1720303803
155,155,sao miguel do araguaia,-13.2750,-50.1628,26.50,31,0,1.19,BR,1720303934
205,205,buka,40.8108,69.1986,22.46,30,0,1.24,UZ,1720304000
207,207,g'uzor,38.6214,66.2522,21.02,29,0,1.11,UZ,1720304002
209,209,bismarck,46.8083,-100.7837,25.36,51,0,2.06,US,1720303778
228,228,sona,45.4388,10.8255,24.69,83,0,0.51,IT,1720304030
297,297,beira,-19.8436,34.8389,21.05,94,0,2.06,MZ,1720304122
303,303,uruzgan,32.8333,66.0000,22.48,23,0,3.22,AF,1720304130
327,327,filiatra,37.1500,21.5833,21.64,65,0,3.35,GR,1720304160


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
cols_to_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = city_data_df[cols_to_copy].copy(deep = True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,kandahar,AF,31.0000,65.7500,14,
116,prince george,CA,53.9166,-122.7530,41,
155,sao miguel do araguaia,BR,-13.2750,-50.1628,31,
205,buka,UZ,40.8108,69.1986,30,
207,g'uzor,UZ,38.6214,66.2522,29,
209,bismarck,US,46.8083,-100.7837,51,
228,sona,IT,45.4388,10.8255,83,
297,beira,MZ,-19.8436,34.8389,94,
303,uruzgan,AF,32.8333,66.0000,23,
327,filiatra,GR,37.1500,21.5833,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kandahar - nearest hotel: No hotel found
prince george - nearest hotel: Travelodge by Wyndham Prince George
sao miguel do araguaia - nearest hotel: Hotel São Miguel
buka - nearest hotel: No hotel found
g'uzor - nearest hotel: No hotel found
bismarck - nearest hotel: Radisson Hotel Bismarck
sona - nearest hotel: Al Bosco
beira - nearest hotel: Hotel Savoy
uruzgan - nearest hotel: No hotel found
filiatra - nearest hotel: Hotel Limenari
smithers - nearest hotel: Sunshine Inn Hotel
huanggang - nearest hotel: 聚贤园迎宾馆
beni saf - nearest hotel: Hôtel Aquarium
al matariyah - nearest hotel: No hotel found
tinghir - nearest hotel: Riad Agraw
grenada - nearest hotel: Allegro Granada
fort st. john - nearest hotel: Fort St. John Motor Inn
ginosa - nearest hotel: Il Borgo Ritrovato


,City,Country,Lat,Lng,Humidity,Hotel Name
11,kandahar,AF,31.0000,65.7500,14,No hotel found
116,prince george,CA,53.9166,-122.7530,41,Travelodge by Wyndham Prince George
155,sao miguel do araguaia,BR,-13.2750,-50.1628,31,Hotel São Miguel
205,buka,UZ,40.8108,69.1986,30,No hotel found
207,g'uzor,UZ,38.6214,66.2522,29,No hotel found
209,bismarck,US,46.8083,-100.7837,51,Radisson Hotel Bismarck
228,sona,IT,45.4388,10.8255,83,Al Bosco
297,beira,MZ,-19.8436,34.8389,94,Hotel Savoy
303,uruzgan,AF,32.8333,66.0000,23,No hotel found
327,filiatra,GR,37.1500,21.5833,65,Hotel Limenari


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity',
    frame_width = 700,
    frame_height = 500,
    scale = 0.75,
    color = 'City',
    alpha = 0.5,
    hover_cols = ["Country", "Hotel Name"]
)

hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)